## Workflow lessons

In [1]:
# Import libraries
import warnings
from glob import glob
import os

import numpy as np
import numpy.ma as ma
import pandas as pd
import matplotlib.pyplot as plt
#from matplotlib import patches as mpatches
from matplotlib.colors import ListedColormap
import matplotlib as mpl
import re  # regular expressions

import rasterio as rio
from rasterio.plot import plotting_extent
#from rasterio.mask import mask
import geopandas as gpd
#from shapely.geometry import mapping

import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep
import earthpy.mask as em


os.chdir(os.path.join(et.io.HOME, 'earth-analytics', 'data'))
warnings.simplefilter('ignore')

## Import Landsat Data For Cold Springs

In the cell or cells below open and process your Landsat Data

In [2]:
# Get Landsat data
et.data.get_data(url="https://ndownloader.figshare.com/files/21941085")

'/Users/leahwasser/earth-analytics/data/earthpy-downloads/landsat-coldsprings-hw'

## Start With Pseudocoding

Coding can seem daunting when you are starting from scratch. Start with what you 
know - English or whatever your native language is! Write out the steps as comments
FIRST. 

In this case, You know that you have two directories of landsat scenes to process in this case.
This will likely mean you will want to loop through and process each scene. 


In [3]:
# To begin create your pseudo code

# High level

# First create the high level loop to loop through each landsat directory of tif files (each scene)
# Get all directories  - remember that getting a list makes things more scalabe
# Loop through this list and
# get all tif files in the directory
# open / crop the files
# stack the files - in this case you will want to use many different bands for the 2 veg indices

# Mask the data (cloud mask)

# Calculate NDVI

# Calculate NBR

### Testing Paths with Empty Loops
Let's start with an empty loop. You could also start with the code but i like 
starting with an EMPTY loop to ensure I can access my directories properly! 

In [4]:
data_path = os.path.join("earthpy-downloads", "landsat-coldsprings-hw")
# view subdirectories of data
all_dirs = glob(data_path + "/*/")

# Open fire boundary
fire_boundary_path = os.path.join("cold-springs-fire",
                                  "vector_layers",
                                  "fire-boundary-geomac",
                                  "co_cold_springs_20160711_2200_dd83.shp")
fire_boundary = gpd.read_file(fire_boundary_path)

# Test that the loop works as you think it should on the directories
for landsat_dir in all_dirs:
    all_files = []
    print(landsat_dir)
    # Get all tif files in the directory
    all_files = sorted(glob(landsat_dir + "/*.tif"))

earthpy-downloads/landsat-coldsprings-hw/LC080340322016062101T1-SC20200306230017/
earthpy-downloads/landsat-coldsprings-hw/LC080340322016072301T1-SC20200306230205/


### Parsing Paths to Extract Metadata

Ok great - my loop is printing out two paths to the two scenes that i want to 
process. things are so far working as i'd like them to. 

In [5]:
# Test that the loop works as you think it should on the directories
# Begin to collect metadata that you will need from the path / file names
for landsat_dir in all_dirs:
    all_files = []
    print("path: ", landsat_dir)
    # Get the landsat scene name / directory
    landsat_scene = os.path.basename(os.path.normpath(landsat_dir))
    print("scene name: ", landsat_scene)
    # Get the date from the scene name
    print("date: ", landsat_scene[10:18])
    # Create list of all tif files in the directory
    all_files = sorted(glob(landsat_dir + "/*.tif"))
    print("\n")

all_files

path:  earthpy-downloads/landsat-coldsprings-hw/LC080340322016062101T1-SC20200306230017/
scene name:  LC080340322016062101T1-SC20200306230017
date:  20160621


path:  earthpy-downloads/landsat-coldsprings-hw/LC080340322016072301T1-SC20200306230205/
scene name:  LC080340322016072301T1-SC20200306230205
date:  20160723




['earthpy-downloads/landsat-coldsprings-hw/LC080340322016072301T1-SC20200306230205/LC08_L1TP_034032_20160723_20180131_01_T1_pixel_qa.tif',
 'earthpy-downloads/landsat-coldsprings-hw/LC080340322016072301T1-SC20200306230205/LC08_L1TP_034032_20160723_20180131_01_T1_radsat_qa.tif',
 'earthpy-downloads/landsat-coldsprings-hw/LC080340322016072301T1-SC20200306230205/LC08_L1TP_034032_20160723_20180131_01_T1_sr_aerosol.tif',
 'earthpy-downloads/landsat-coldsprings-hw/LC080340322016072301T1-SC20200306230205/LC08_L1TP_034032_20160723_20180131_01_T1_sr_band1.tif',
 'earthpy-downloads/landsat-coldsprings-hw/LC080340322016072301T1-SC20200306230205/LC08_L1TP_034032_20160723_20180131_01_T1_sr_band2.tif',
 'earthpy-downloads/landsat-coldsprings-hw/LC080340322016072301T1-SC20200306230205/LC08_L1TP_034032_20160723_20180131_01_T1_sr_band3.tif',
 'earthpy-downloads/landsat-coldsprings-hw/LC080340322016072301T1-SC20200306230205/LC08_L1TP_034032_20160723_20180131_01_T1_sr_band4.tif',
 'earthpy-downloads/land

### Add Code to Your PseudoCode
Now you can start to work through adding code to your pseudocode. Take it 
step by step and think about potential issues that you may encounter in each step.

Below you can see the code filled in - in between each comment which comes from the pseudo code.

Developing the example below was a process:

1. Create your pseudo code for the steps that you want to perform 
2. Create your EMPTY outer loop as a test to implement this across multiple directories (above)
3. Begin to add code below each line of pseudo code. as you add your code:
    1. Consider the inputs and ouputs of each function (use help(function_name) to help with this. 
    2. Consider where things could fail and add tests using try: statements or conditionals (`if` statements)
    
Now you have a few options. You can create the loop first and add code incrementally.
You could also create the loop AFTEr writing out your code. 

A nice middle ground is to create your loop but only loop through one directory to begin with. 
Create your workflow. Then combine the two

In [48]:
data_path = os.path.join("earthpy-downloads", "landsat-coldsprings-hw")

# First create the high level loop to loop through each landsat directory of tif files (each scene)

# Get all directories  - remember that getting a list makes things more scalabe
all_dirs = glob(data_path + "/*/")

# Create a loop variable that you can use to run your code for a single directory
landsat_dir = all_dirs[0]
# Loop through this list - commenting out the loop for now and just focusing on the workflow! 
#for landsat_dir in all_dirs:
all_files = []

# Get all tif files in the directory
all_files = sorted(glob(landsat_dir + "/*.tif"))
crop_path = os.path.join(landsat_dir, "cropped")

# open / crop the files - this actually requires several steps including
# 1. creating an output crop path,
# 2. opening up the fire boundary to crop
# 3. Cropping the data
if not os.path.exists(crop_path):
    os.mkdir(crop_path)

with rio.open(all_files[0]) as src:
    fire_bound_reproject = fire_boundary.to_crs(src.crs)

es.crop_all(raster_paths=all_files,
            output_dir=crop_path,
            geoms=fire_bound_reproject,
            overwrite=True)
# Get a list of bands to stack
sorted(glob(crop_path + "/*band*.tif"))

# Stack the bands for NDVI and NBR calculation


# Mask clouds from the stacked bands

# 1. open the qa layer


# Generate list of values to mask using the pixel qa layer


# 2. mask the numpy array
# Create test to ensure the values are in the mask qa layer

# Perform the steps below for each landsat scene
# Because we are going to need several bands to calculate NDVI and NBR let's crop them all with es.crop_all


# Clean data (clouds and no data values)

# Calculate nbr

# Calculate dNBR from pre-post fire nbr

# Reclassify dnbr to create final plot

# Plot data

['earthpy-downloads/landsat-coldsprings-hw/LC080340322016062101T1-SC20200306230017/cropped/LC08_L1TP_034032_20160621_20170221_01_T1_sr_band1_crop.tif',
 'earthpy-downloads/landsat-coldsprings-hw/LC080340322016062101T1-SC20200306230017/cropped/LC08_L1TP_034032_20160621_20170221_01_T1_sr_band2_crop.tif',
 'earthpy-downloads/landsat-coldsprings-hw/LC080340322016062101T1-SC20200306230017/cropped/LC08_L1TP_034032_20160621_20170221_01_T1_sr_band3_crop.tif',
 'earthpy-downloads/landsat-coldsprings-hw/LC080340322016062101T1-SC20200306230017/cropped/LC08_L1TP_034032_20160621_20170221_01_T1_sr_band4_crop.tif',
 'earthpy-downloads/landsat-coldsprings-hw/LC080340322016062101T1-SC20200306230017/cropped/LC08_L1TP_034032_20160621_20170221_01_T1_sr_band5_crop.tif',
 'earthpy-downloads/landsat-coldsprings-hw/LC080340322016062101T1-SC20200306230017/cropped/LC08_L1TP_034032_20160621_20170221_01_T1_sr_band6_crop.tif',
 'earthpy-downloads/landsat-coldsprings-hw/LC080340322016062101T1-SC20200306230017/cropp

In [49]:
data_path = os.path.join("earthpy-downloads", "landsat-coldsprings-hw")

# First create the high level loop to loop through each landsat directory of tif files (each scene)

# Get all directories  - remember that getting a list makes things more scalabe
all_dirs = glob(data_path + "/*/")

# Loop through this list

all_files = []
# print(landsat_dir)
# Get all tif files in the directory
all_files = sorted(glob(landsat_dir + "/*.tif"))
crop_path = os.path.join(landsat_dir, "cropped")

# open / crop the files - this actually requires several steps including
# 1. creating an output crop path,
# 2. opening up the fire boundary to crop
# 3. Cropping the data
if not os.path.exists(crop_path):
    os.mkdir(crop_path)

with rio.open(all_files[0]) as src:
    fire_bound_reproject = fire_boundary.to_crs(src.crs)

es.crop_all(raster_paths=all_files,
            output_dir=crop_path,
            geoms=fire_bound_reproject,
            overwrite=True)
# Get a list of bands to stack
all_bands = sorted(glob(crop_path + "/*band*.tif"))

# Stack the bands for NDVI and NBR calculation
landsat_bands, landsat_meta = es.stack(all_bands)

# Mask clouds from the stacked bands

# 1. open the qa layer
pixel_qa_path = glob(crop_path + "/*pixel_qa*.tif")
with rio.open(pixel_qa_path[0]) as src:
    mask_arr = src.read(1)

# Generate list of values to mask using the pixel qa layer
high_cloud_confidence = em.pixel_flags["pixel_qa"]["L8"]["High Cloud Confidence"]
cloud = em.pixel_flags["pixel_qa"]["L8"]["Cloud"]
cloud_shadow = em.pixel_flags["pixel_qa"]["L8"]["Cloud Shadow"]
all_masked_values = cloud_shadow + cloud + high_cloud_confidence

# 2. mask the numpy array
# Create test to ensure the values are in the mask qa layer
if any(i in np.unique(mask_arr) for i in all_masked_values):
    landsat_masked_bands = em.mask_pixels(landsat_bands,
                                          mask_arr,
                                          vals=all_masked_values)
else:
    print("There are no pixels to mask")

# Perform the steps below for each landsat scene
# Because we are going to need several bands to calculate NDVI and NBR let's crop them all with es.crop_all


# Clean data (clouds and no data values)

# Calculate nbr

# Calculate dNBR from pre-post fire nbr

# Reclassify dnbr to create final plot

# Plot data

There are no pixels to mask


Once you have setup your code and it works on a scene, or within your loop you 
can think about making it modular. I often look at my code and find steps that 
are related that might pair nicely together in a function. Well-named Functions will 
make your code easier to read and trouble shoot.

In [ ]:
data_path = os.path.join("earthpy-downloads", "landsat-coldsprings-hw")

# First create the high level loop to loop through each landsat directory of tif files (each scene)

# Get all directories  - remember that getting a list makes things more scalabe
all_dirs = glob(data_path + "/*/")

# Loop through this list


all_files = []
# print(landsat_dir)
# Get all tif files in the directory
all_files = sorted(glob(landsat_dir + "/*.tif"))
crop_path = os.path.join(landsat_dir, "cropped")

# BEGIN FUNCTION that crops and returns a cropped stacked array of bands
# open / crop the files - this actually requires several steps including
# 1. creating an output crop path,
# 2. opening up the fire boundary to crop
# 3. Cropping the data
if not os.path.exists(crop_path):
    os.mkdir(crop_path)

with rio.open(all_files[0]) as src:
    fire_bound_reproject = fire_boundary.to_crs(src.crs)

es.crop_all(raster_paths=all_files,
            output_dir=crop_path,
            geoms=fire_bound_reproject,
            overwrite=True)
# Get a list of bands to stack
all_bands = sorted(glob(crop_path + "/*band*.tif"))

# Stack the bands for NDVI and NBR calculation
landsat_bands, landsat_meta = es.stack(all_bands)
# END FUNCTION that crops and returns a stack...

# BEGIN MASK FUNCTION
# Mask clouds from the stacked bands

# 1. open the qa layer
pixel_qa_path = glob(crop_path + "/*pixel_qa*.tif")
with rio.open(pixel_qa_path[0]) as src:
    mask_arr = src.read(1)

# Cloud mask values
high_cloud_confidence = em.pixel_flags["pixel_qa"]["L8"]["High Cloud Confidence"]
cloud = em.pixel_flags["pixel_qa"]["L8"]["Cloud"]
cloud_shadow = em.pixel_flags["pixel_qa"]["L8"]["Cloud Shadow"]

all_masked_values = cloud_shadow + cloud + high_cloud_confidence
# 2. mask the numpy array
# Create test to ensure the values are in the mask qa layer
if any(i in np.unique(mask_arr) for i in all_masked_values):
    landsat_masked_bands = em.mask_pixels(landsat_bands,
                                          mask_arr,
                                          vals=all_masked_values)
else:
    print("There are no pixels to mask")
# END MASK FUNCTION


# Perform the steps below for each landsat scene
# Because we are going to need several bands to calculate NDVI and NBR let's crop them all with es.crop_all


# Clean data (clouds and no data values)

# Calculate nbr

# Calculate dNBR from pre-post fire nbr

# Reclassify dnbr to create final plot

# Plot data

In [ ]:
data_path = os.path.join("earthpy-downloads", "landsat-coldsprings-hw")

# First create the high level loop to loop through each landsat directory of tif files (each scene)

# Get all directories  - remember that getting a list makes things more scalabe
all_dirs = glob(data_path + "/*/")

# Loop through this list

for landsat_dir in all_dirs:
    all_files = []
    # print(landsat_dir)
    # Get all tif files in the directory
    all_files = sorted(glob(landsat_dir + "/*.tif"))
    crop_path = os.path.join(landsat_dir, "cropped")

    # BEGIN FUNCTION that crops and returns a cropped stacked array of bands
    # open / crop the files - this actually requires several steps including
    # 1. creating an output crop path,
    # 2. opening up the fire boundary to crop
    # 3. Cropping the data
    if not os.path.exists(crop_path):
        os.mkdir(crop_path)

    with rio.open(all_files[0]) as src:
        fire_bound_reproject = fire_boundary.to_crs(src.crs)

    es.crop_all(raster_paths=all_files,
                output_dir=crop_path,
                geoms=fire_bound_reproject,
                overwrite=True)
    # Get a list of bands to stack
    all_bands = sorted(glob(crop_path + "/*band*.tif"))

    # Stack the bands for NDVI and NBR calculation
    landsat_bands, landsat_meta = es.stack(all_bands)
    # END FUNCTION that crops and returns a stack...

    # BEGIN MASK FUNCTION
    # Mask clouds from the stacked bands

    # 1. open the qa layer
    pixel_qa_path = glob(crop_path + "/*pixel_qa*.tif")
    with rio.open(pixel_qa_path[0]) as src:
        mask_arr = src.read(1)

    # Cloud mask values
    high_cloud_confidence = em.pixel_flags["pixel_qa"]["L8"]["High Cloud Confidence"]
    cloud = em.pixel_flags["pixel_qa"]["L8"]["Cloud"]
    cloud_shadow = em.pixel_flags["pixel_qa"]["L8"]["Cloud Shadow"]

    all_masked_values = cloud_shadow + cloud + high_cloud_confidence
    # 2. mask the numpy array
    # Create test to ensure the values are in the mask qa layer
    if any(i in np.unique(mask_arr) for i in all_masked_values):
        landsat_masked_bands = em.mask_pixels(landsat_bands,
                                              mask_arr,
                                              vals=all_masked_values)
    else:
        print("There are no pixels to mask")
    # END MASK FUNCTION


# Perform the steps below for each landsat scene
# Because we are going to need several bands to calculate NDVI and NBR let's crop them all with es.crop_all


# Clean data (clouds and no data values)

# Calculate nbr

# Calculate dNBR from pre-post fire nbr

# Reclassify dnbr to create final plot

# Plot data

In [ ]:
def crop_stack_data(files_to_crop, crop_path, crop_bound):
    """Crops a set of tif files and saves them in a crop directory.
    Returns a stacked numpy array of bands"""
    if not os.path.exists(crop_path):
        os.mkdir(crop_path)

    with rio.open(all_files[0]) as src:
        fire_bound_reproject = fire_boundary.to_crs(src.crs)

    es.crop_all(raster_paths=all_files,
                output_dir=crop_path,
                geoms=fire_bound_reproject,
                overwrite=True)
    # Get a list of bands to stack
    all_bands = sorted(glob(crop_path + "/*band*.tif"))

    # Stack the bands for NDVI and NBR calculation
    return es.stack(all_bands)


def mask_data(arr, path_to_qa):
    """Function that masks a numpy array using a cloud qa layer"""

    # 1. open the qa layer
    with rio.open(pixel_qa_path[0]) as src:
        mask_arr = src.read(1)

    # Cloud mask values
    high_cloud_confidence = em.pixel_flags["pixel_qa"]["L8"]["High Cloud Confidence"]
    cloud = em.pixel_flags["pixel_qa"]["L8"]["Cloud"]
    cloud_shadow = em.pixel_flags["pixel_qa"]["L8"]["Cloud Shadow"]

    all_masked_values = cloud_shadow + cloud + high_cloud_confidence
    # 2. mask the numpy array
    # Create test to ensure the values are in the mask qa layer
    if any(i in np.unique(mask_arr) for i in all_masked_values):
        landsat_masked_bands = em.mask_pixels(landsat_bands,
                                              mask_arr,
                                              vals=all_masked_values)
        return landsat_masked_bands
    else:
        print("There are no pixels to mask")
        return arr

After creating the functions notice how much easier the code is to read!
At this point you have a working loop and you can continue to add pieces of 
your workflow.

Functions are a nice way to organize your work because they allow you to 
modularize your workflow, extracting steps into individual components that you 
can test. 

In [ ]:
data_path = os.path.join("earthpy-downloads", "landsat-coldsprings-hw")
# view subdirectories of data
all_dirs = glob(data_path + "/*/")


#####
# High level

# First create the high level loop to loop through 1) each landsat directory and then 2) the files
for landsat_dir in all_dirs:
    all_files = []
    # print(landsat_dir)
    # Get all tif files in the directory
    all_files = sorted(glob(landsat_dir + "/*.tif"))
    crop_path = os.path.join(landsat_dir, "cropped")

    # crop & stack data - note that the no data values have NOT been accounted for yet
    landsat_arr, landsat_meta = crop_stack_data(files_to_crop=all_files,
                                                crop_path=crop_path,
                                                crop_bound=fire_boundary)

    # Mask data
    pixel_qa_path = glob(crop_path + "/*pixel_qa*.tif")
    landsat_mask = mask_data(landsat_arr, pixel_qa_path)


# Calculate nbr

# Calculate dNBR from pre-post fire nbr

# Reclassify dnbr to create final plot

# Plot data


# Get list of all tif files

Now the code is simpler however there is a remaining issue. the loop processes 
the data for each site but it overwrites the resulting data each time which 
means that you will not be able to calculate NDVI difference. 

Here you have a few options.

1. Create a list of masked arrays and use this list: This could work well and it will scale. It will however require you to keep track of the array index in the list so keep that in mind.  
2. If your loop is small you could create custom variable names for each returned array.
3. You could write out intermediate stacked tif files if you want. Sometimes this option is nice when you have a big workflow that may need to be "Started" somewhere in the middle at some point rather than from the beginning.
4. Create a dictionary with a custom key name (this is an ideal option).

In [ ]:
# First create the high level loop to loop through 1) each landsat directory and then 2) the files
landsat_data = {}

for landsat_dir in all_dirs:
    all_files = []
    # print(landsat_dir)
    # Get all tif files in the directory
    all_files = sorted(glob(landsat_dir + "/*.tif"))
    crop_path = os.path.join(landsat_dir, "cropped")

    # crop & stack data
    landsat_arr, landsat_meta = crop_stack_data(files_to_crop=all_files,
                                                crop_path=crop_path,
                                                crop_bound=fire_boundary)

    # Mask data
    pixel_qa_path = glob(crop_path + "/*pixel_qa*.tif")

    # Get dir name - you could also chose to just use the date as the scene dictionary key
    
    landsat_scene = os.path.basename(os.path.normpath(landsat_dir))
    scene_date = landsat_scene[10:18]

    landsat_data[scene_date] = mask_data(landsat_arr, pixel_qa_path)


landsat_data

In [ ]:
# Plot the data as a visual check
ep.plot_bands(landsat_data["20160723"])
plt.show()

At this point you have a working loop that processes, crops, and stacks your 
data. You can at any time add additional cleaning steps to existing functions 
or you could create new functions and add those to the workflow. 

Next, you may want to calculate your vegetation indices using functions or steps.
Below two additional objects are created for ndvi and nbr respectively. Note that 
you could combine these into one dict or keep them separate depending upon 
your workflow! Or maybe you decided that you do'nt need the dict of cleaned 
data. you ONLY want the dict for NDVI and NBR

In [ ]:
# Nested dictionary can be used to store information hierarchically as an option

veg_indices = {}

veg_indices["date"] = {"ndvi": .8, "nbr": .8}
                       
veg_indices["date"]["ndvi"] 

Below you create a nested dictionary to store information. You can then use this to calculate
your difference raster which you need for the final plots in this assignment.

In [ ]:
# First create the high level loop to loop through 1) each landsat directory and then 2) the files
landsat_data = {}
veg_index = {}

for landsat_dir in all_dirs:
    all_files = []
    # print(landsat_dir)
    # Get all tif files in the directory
    all_files = sorted(glob(landsat_dir + "/*.tif"))
    crop_path = os.path.join(landsat_dir, "cropped")

    # crop & stack data
    landsat_arr, landsat_meta = crop_stack_data(files_to_crop=all_files,
                                                crop_path=crop_path,
                                                crop_bound=fire_boundary)

    # Mask data
    pixel_qa_path = glob(crop_path + "/*pixel_qa*.tif")

    # Get dir name - you could also chose to just use the date as the scene dictionary key
    
    landsat_scene = os.path.basename(os.path.normpath(landsat_dir))
    scene_date = landsat_scene[10:18]

    cleaned_landsat = mask_data(landsat_arr, pixel_qa_path)
    
    # Calculate NDVI - note that the way this is written, it will only work for 
    # landsat / ndvi as the band numbers are hard coded.
    # Also note that the bands below are not necessarily correct.
    veg_index[scene_date] = {"ndvi": es.normalized_diff(cleaned_landsat[4], cleaned_landsat[3]),
                       "nbr": (cleaned_landsat[4], cleaned_landsat[3])} 
    
    # Calculate difference


veg_index

In [ ]:
ep.plot_bands(veg_index["20160723"]["ndvi"],
             scale=False, cmap = "Greens")
plt.show()